In [1]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers
from tqdm import tqdm
from nltk.corpus import wordnet
import models
import nltk
# nltk.download('sentiwordnet')

/home/foivos/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def do_preprocessing(filepath, test_file=False):
    with open(filepath, 'r') as f_in:
        lines = f_in.readlines()
        f_in.close()    
    
    if test_file:
        lines = [line.split(',', 1)[1] for line in lines]
    
    processed_list = []
    for line in tqdm(lines):
        pro_line = line
        pro_line = preproc.remove_new_line(pro_line)
        pro_line = preproc.replace_slang(pro_line)
        pro_line = preproc.replace_contraction(pro_line)        
        pro_line = preproc.remove_tags(pro_line)
        pro_line = preproc.replace_emoji(pro_line)
        pro_line = preproc.convert_to_lowercase(pro_line)
#         no        
#         pro_line = preproc.lemmatize_verbs(pro_line)
        # pro_line = preproc.stemming_using_Porter(pro_line)
#         pro_line = preproc.remove_stopwords(pro_line)
        
#         yes    
        
        pro_line = preproc.separate_hashtags(pro_line)
        pro_line = preproc.replace_numbers(pro_line)
        pro_line = preproc.replace_elongated(pro_line)
        pro_line = preproc.add_sentiment(pro_line)
        processed_list.append(pro_line)

    filename = filepath.split('/')[-1][:-4] + '_processed'
    helpers.write_file(processed_list, filename)

In [3]:
do_preprocessing('../twitter-datasets/train_pos_full.txt')
do_preprocessing('../twitter-datasets/train_neg_full.txt')

100%|██████████| 1250000/1250000 [25:01<00:00, 832.46it/s] 


In [4]:
# lines[0]

In [5]:
start_time = time.time()

In [6]:
# False to return the processed small dataset, True for the big one
lines, y = preproc.return_processed_trainset_and_y(True)

len(lines)

2500000

In [7]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=None)
X = vectorizer.fit_transform(lines)

In [8]:
clf = models.return_model('LR')
clf.fit(X,y)
# kf = KFold(n_splits=4, shuffle=True, random_state=1)
# scores = cross_val_score(clf, X, y, cv=kf, n_jobs=-1)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

------
Elapsed time: 4 min 33 sec



In [10]:
do_preprocessing('../twitter-datasets/test_data.txt')

with open('../twitter-datasets/test_data_processed.txt') as test_in:
    test_lines = test_in.readlines()

print(len(test_lines))

X_test = vectorizer.transform(test_lines)

y_pred = clf.predict(X_test)

helpers.create_submission_csv(y_pred)

print(np.unique(y_pred))

100%|██████████| 10000/10000 [00:11<00:00, 888.32it/s]


10000
[-1.  1.]
